In [3]:
import google.generativeai as genai
import weave

import os
from dotenv import load_dotenv

def format_res(text):
    return text.replace('•', '  *')
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [6]:
query = "what can you do?"

model = genai.GenerativeModel('gemini-2.0-flash-exp')
project_name = "roboweave"
weave.init(project_name)


In [17]:
@weave.op
def query_model(query: str):
    response = model.generate_content(
        query
    )
    return format_res(response.text)



In [18]:
query_model("what can you do?")

"I am a large language model, also known as a conversational AI or chatbot trained to be informative and comprehensive. I am trained on a massive amount of text data and some image, audio and video data.\n\nHere's a breakdown of what I can do:\n\n**Text-Based Tasks:**\n\n*   **Generate Text:** I can write different kinds of creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I will try my best to follow all your requirements.\n*   **Answer Questions:** I can answer questions in a comprehensive and informative way, even if they are open-ended, challenging, or strange. I will use my knowledge to provide helpful, relevant, and accurate responses.\n*   **Translate Languages:** I can translate text from one language to another.\n*   **Summarize Text:** I can provide concise summaries of long articles, reports, or documents.\n*   **Write Different Kinds of Content:** I can help you create content for various purposes, such as blog posts, articles, social me